<a href="https://colab.research.google.com/github/simon-clematide/colab-notebooks-for-teaching/blob/main/notebooks/topic_modeling_sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup




In [ ]:
#! pip install --upgrade pandas-profiling jupyter
! pip install pyldavis==3.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# avoid warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# `pyLDAvis`

pyLDAvis now also supports LDA application from scikit-learn. Let's take a look into this in more detail. We will be using the 20 newsgroups dataset as provided by scikit-learn.

In [ ]:
import pyLDAvis
import pyLDAvis.lda_model
pyLDAvis.enable_notebook()

## Load 20 newsgroups dataset

First, the 20 newsgroups dataset available in sklearn is loaded. As always, the headers, footers and quotes are removed.

Newsgroup categories:
`['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']`

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
cats = ['alt.atheism', 'talk.politics.guns', 'rec.autos', 'sci.space','rec.sport.baseball']
newsgroups = fetch_20newsgroups(remove=('headers', 'footers', 'quotes'),categories=cats)
docs_raw = newsgroups.data
print(len(docs_raw))

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


2810


In [ ]:
print(docs_raw[72])

An excellent reference for non-technical readers on the ORION system is
"The Starflight Handbook", by Eugene Mallove and Gregory Matloff, ISBN
0-471-61912-4. The relevant chapter is 4: Nuclear Pulse Propulsion.

The book also contains lots of technical references for the more academically
inclined. 

Enjoy!


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Convert to document-term matrix

Next, the raw documents are converted into document-term matrix, possibly as raw counts or in TF-IDF form.

In [ ]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5,  # exclude words with a relative document frequency greater than 50%
                                min_df = 10    # exclude tokens that occur less than 10 times
                                )
dtm_tf = tf_vectorizer.fit_transform(docs_raw)
print(dtm_tf.shape)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(2810, 3242)


In [ ]:
# How does a certain document look like in this representation?
# Get the mapping of column indices to vocabulary items
index2vocabulary_item = tf_vectorizer.get_feature_names_out()

# Get the dense matrix representation of the document-term matrix
doc_index = 72  # Index of the document to show
doc_matrix = dtm_tf.getrow(doc_index).toarray()

# Print the words and their counts in the document
for i, count in enumerate(doc_matrix[0]):
    if count > 0:
        word = index2vocabulary_item[i]
        print(f"{word}: {count}")

book: 1
chapter: 1
contains: 1
enjoy: 1
excellent: 1
handbook: 1
lots: 1
non: 1
nuclear: 1
propulsion: 1
readers: 1
reference: 1
references: 1
relevant: 1
technical: 2


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Alternative, we can build a tf-idf document-term matrix

In [ ]:
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(docs_raw)
print(dtm_tfidf.shape)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(2810, 3242)


In [ ]:
# How does a certain document look like in this representation?
# Get the mapping of column indices to vocabulary items
index2vocabulary_item = tfidf_vectorizer.get_feature_names_out()

# Get the dense matrix representation of the document-term matrix
doc_index = 72  # Index of the document to show
doc_matrix = dtm_tfidf.getrow(doc_index).toarray()

# Print the words and their counts in the document
for i, count in enumerate(doc_matrix[0]):
    if count > 0:
        word = index2vocabulary_item[i]
        print(f"{word}: {count}")

book: 0.19712805455436383
chapter: 0.2681673200324737
contains: 0.25239101617248827
enjoy: 0.25634576042578194
excellent: 0.21951184984071434
handbook: 0.27397222478460365
lots: 0.22132345405314058
non: 0.18637993738956596
nuclear: 0.23941486626728636
propulsion: 0.2543234325332879
readers: 0.25634576042578194
reference: 0.23056396971345594
references: 0.24231408770985763
relevant: 0.23941486626728636
technical: 0.44264690810628116


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Fit Latent Dirichlet Allocation models

Finally, the LDA models are fitted. n_components is number of topics.

In [ ]:
# for TF DTM
lda_tf = LatentDirichletAllocation(n_components=10, random_state=0,verbose=1, max_iter=10)
lda_tf.fit(dtm_tf)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


LatentDirichletAllocation(random_state=0, verbose=1)

In [ ]:
n

## Visualizing the models with pyLDAvis
Multidimensional scaling = Dimension reduction

Can you reidentify the newsgroups? `['alt.atheism', 'talk.politics.guns', 'rec.autos', 'sci.space','rec.sport.baseball']`

Hover over topics circles and terms to explore the connection between words and topics...

In [ ]:
pyLDAvis.lda_model.prepare(lda_tf, dtm_tf, tf_vectorizer)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.133141 -0.052176       1        1  20.436474
7     -0.014638 -0.242254       2        1  14.398071
6     -0.140931  0.093976       3        1  13.433040
4     -0.149383 -0.008567       4        1  11.173464
1     -0.075872  0.083069       5        1   9.005669
5      0.156900 -0.002867       6        1   8.536595
9     -0.047921  0.121986       7        1   6.642455
3      0.226752  0.104384       8        1   6.472224
0      0.248759 -0.051613       9        1   5.962893
2     -0.070524 -0.045937      10        1   3.939115, topic_info=         Term        Freq        Total Category  logprob  loglift
2732    space  993.000000   993.000000  Default  30.0000  30.0000
413       car  551.000000   551.000000  Default  29.0000  29.0000
1256      gun  620.000000   620.000000  Default  28.0000  28.0000
1893     nasa  382.000000   382.000000  Default  27.0000  27.0000
1215      god  429.000000   429.000000  Default  26.0000  26.0000
...       ...         ...          ...      ...      ...      ...
1193  general   26.771323   199.191135  Topic10  -5.4507   1.2273
2745    speed   24.840730   142.583436  Topic10  -5.5255   1.4868
853       don   26.769855  1004.736248  Topic10  -5.4507  -0.3910
2547     said   24.373420   393.551467  Topic10  -5.5445   0.4525
1563     know   23.726384   659.486795  Topic10  -5.5714  -0.0907

[637 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
4         3  0.076718      abs
4         5  0.869466      abs
4         9  0.025573      abs
40        2  0.208383  address
40        4  0.072934  address
...     ...       ...      ...
3237      3  0.195699     york
3237      5  0.010300     york
3237      6  0.185399     york
3237      7  0.463497     york
3237      9  0.041200     york

[1945 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 8, 7, 5, 2, 6, 10, 4, 1, 3])

## Topix Modeling with TFIDF values

In [ ]:
# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=10, random_state=0, verbose=1,max_iter=10)
lda_tfidf.fit(dtm_tfidf)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


LatentDirichletAllocation(random_state=0, verbose=1)

In [ ]:
pyLDAvis.lda_model.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.102076 -0.000023       1        1  32.181358
9      0.121331 -0.108234       2        1  26.835350
5      0.049222  0.174625       3        1  20.801335
3     -0.186419 -0.020705       4        1   6.767724
4     -0.016382 -0.015039       5        1   2.639377
2     -0.015609 -0.011735       6        1   2.298800
7     -0.013186 -0.008824       7        1   2.203791
1     -0.017008  0.000561       8        1   2.191786
8     -0.013779 -0.009607       9        1   2.137860
0     -0.010246 -0.001020      10        1   1.942618, topic_info=         Term       Freq      Total Category  logprob  loglift
2732    space  32.000000  32.000000  Default  30.0000  30.0000
413       car  37.000000  37.000000  Default  29.0000  29.0000
1655     list  14.000000  14.000000  Default  28.0000  28.0000
901       edu  19.000000  19.000000  Default  27.0000  27.0000
1215      god  22.000000  22.000000  Default  26.0000  26.0000
...       ...        ...        ...      ...      ...      ...
76        air   0.563414   8.923309  Topic10  -6.1213   1.1787
1700     luck   0.493166   5.542279  Topic10  -6.2545   1.5218
3232     year   0.517881  32.097189  Topic10  -6.2056  -0.1856
166   article   0.486056  12.547278  Topic10  -6.2690   0.6902
2722     sort   0.471846   9.704115  Topic10  -6.2987   0.9175

[523 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
35        1  0.367444  actually
35        2  0.419936  actually
35        3  0.157476  actually
35        4  0.052492  actually
48        1  0.410820   advance
...     ...       ...       ...
3233      2  0.238459     years
3233      3  0.190768     years
3233      4  0.047692     years
3234      1  0.414391       yep
3234      8  0.414391       yep

[911 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 10, 6, 4, 5, 3, 8, 2, 9, 1])

### Using different MDS functions

With `sklearn` installed, other MDS functions, such as MMDS and TSNE can be used for plotting if the default PCoA is not satisfactory.

In [ ]:
pyLDAvis.lda_model.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='mmds')

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.179795  0.036325       1        1  20.436474
7     -0.280323  0.158074       2        1  14.398071
6      0.046608  0.279297       3        1  13.433040
4     -0.060637  0.164723       4        1  11.173464
1      0.078997  0.006132       5        1   9.005669
5      0.256272 -0.073744       6        1   8.536595
9      0.182012  0.153303       7        1   6.642455
3      0.193031 -0.256820       8        1   6.472224
0     -0.045854 -0.340121       9        1   5.962893
2     -0.190310 -0.127168      10        1   3.939115, topic_info=         Term        Freq        Total Category  logprob  loglift
2732    space  993.000000   993.000000  Default  30.0000  30.0000
413       car  551.000000   551.000000  Default  29.0000  29.0000
1256      gun  620.000000   620.000000  Default  28.0000  28.0000
1893     nasa  382.000000   382.000000  Default  27.0000  27.0000
1215      god  429.000000   429.000000  Default  26.0000  26.0000
...       ...         ...          ...      ...      ...      ...
1193  general   26.771323   199.191135  Topic10  -5.4507   1.2273
2745    speed   24.840730   142.583436  Topic10  -5.5255   1.4868
853       don   26.769855  1004.736248  Topic10  -5.4507  -0.3910
2547     said   24.373420   393.551467  Topic10  -5.5445   0.4525
1563     know   23.726384   659.486795  Topic10  -5.5714  -0.0907

[637 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
4         3  0.076718      abs
4         5  0.869466      abs
4         9  0.025573      abs
40        2  0.208383  address
40        4  0.072934  address
...     ...       ...      ...
3237      3  0.195699     york
3237      5  0.010300     york
3237      6  0.185399     york
3237      7  0.463497     york
3237      9  0.041200     york

[1945 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 8, 7, 5, 2, 6, 10, 4, 1, 3])

In [ ]:
pyLDAvis.lda_model.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='tsne')

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
8      10.899565  16.745018       1        1  20.436474
7      53.879875  24.046295       2        1  14.398071
6      49.288376 -26.045382       3        1  13.433040
4     -71.018761  -2.107758       4        1  11.173464
1       9.541195 -67.383278       5        1   9.005669
5     -38.603615  44.873913       6        1   8.536595
9       4.593139 -24.923311       7        1   6.642455
3     -28.379095   1.237335       8        1   6.472224
0     -39.802765 -46.079662       9        1   5.962893
2       9.319964  60.272034      10        1   3.939115, topic_info=         Term        Freq        Total Category  logprob  loglift
2732    space  993.000000   993.000000  Default  30.0000  30.0000
413       car  551.000000   551.000000  Default  29.0000  29.0000
1256      gun  620.000000   620.000000  Default  28.0000  28.0000
1893     nasa  382.000000   382.000000  Default  27.0000  27.0000
1215      god  429.000000   429.000000  Default  26.0000  26.0000
...       ...         ...          ...      ...      ...      ...
1193  general   26.771323   199.191135  Topic10  -5.4507   1.2273
2745    speed   24.840730   142.583436  Topic10  -5.5255   1.4868
853       don   26.769855  1004.736248  Topic10  -5.4507  -0.3910
2547     said   24.373420   393.551467  Topic10  -5.5445   0.4525
1563     know   23.726384   659.486795  Topic10  -5.5714  -0.0907

[637 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
4         3  0.076718      abs
4         5  0.869466      abs
4         9  0.025573      abs
40        2  0.208383  address
40        4  0.072934  address
...     ...       ...      ...
3237      3  0.195699     york
3237      5  0.010300     york
3237      6  0.185399     york
3237      7  0.463497     york
3237      9  0.041200     york

[1945 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 8, 7, 5, 2, 6, 10, 4, 1, 3])

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
